# 数据清洗

In [5]:
import pandas as pd
import re
import os
import liwc
import jieba
import pickle
from ltp import LTP
from ltp import StnSplit

In [2]:
ltp_model_path = 'ltp_base'
# 初始化LTP模型
ltp = LTP(ltp_model_path)

Loading weights from local directory


In [3]:
df = pd.read_excel('评论数据集/all.xlsx', engine="openpyxl")

In [4]:
df

,text,label
0,排了蛮久 味道不错 不太好找 还会去,0
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0
2,环境很整洁，就是一楼很热，馄饨的量也很足,0
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0
...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1


In [5]:
df['sents'] = None 
df['clean_text'] = None
df['words'] = None 

In [6]:
# 文本清洗
def clean_text(text):
    clean_text = re.sub(r'[^\u4e00-\u9fa5]+', '', text)
    return clean_text

In [7]:
# 分句
import re
def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # 英文省略号
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # 中文省略号
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return para.split("\n")

In [8]:
df['sents'] = df['text'].apply(cut_sent)

In [9]:
df['clean_text'] = df['text'].apply(clean_text)

In [10]:
df

,text,label,sents,clean_text,words
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,None
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,None
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,None
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,None
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,None
...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,None
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,None
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,None
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,None


In [11]:
# 分词
for index, row in df.iterrows():
    text = row['clean_text']

    # Perform word segmentation
    words = jieba.lcut(text)

    # Cache the word segmentation results
    with open(f'words_{index}.pkl', 'wb') as f:
        pickle.dump(words, f)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 0.611 seconds.
Prefix dict has been built successfully.


In [12]:
for index, row in df.iterrows():
    # Load the cached word segmentation results
    with open(f'words_{index}.pkl', 'rb') as f:
        cached_words = pickle.load(f)
    # Update the 'words' column in the DataFrame with the cached results
    df.at[index, 'words'] = cached_words

In [13]:
df

,text,label,sents,clean_text,words
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]"
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量..."
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]"
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,..."
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇..."
...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ..."
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,..."
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比..."
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻..."


In [14]:
df.to_excel('clean_cut.xlsx')

# 特征提取

In [15]:
# 编码处理
import codecs
import jieba
import ltp
from ltp import LTP
from ltp import StnSplit
import pandas as pd


ltp_model_path = 'ltp_base'
# 初始化LTP模型
ltp = LTP(ltp_model_path)


Loading weights from local directory


In [16]:
# 一些处理工具
# 读自定义的词库
def read_txtfile(file_path):
    with open(file_path, 'r', encoding="utf-8") as file:
        return [line.strip() for line in file]


# 读取LIWC词典并计算目标词数量
dict = pd.read_csv("LIWC2015 Dictionary - Chinese (Simplified).dicx",low_memory=False)

dicterm_set = set(dict['DicTerm'])

def count_liwc(words, categories):
    counts = {cat: 0 for cat in categories}
    word_aim = []

    for word in words:
        if word in dicterm_set:  # Use the set for lookup
            row = dict.loc[dict['DicTerm'] == word]
            for cat in categories:
                if not pd.isna(row[cat].values[0]):
                    counts[cat] += 1
                    word_aim.append(word)

    total = sum(counts.values())

    return total, counts, word_aim

## 认知负荷相关特征

### 词数 word count

In [17]:
df['word_count'] = df.words.apply(len)

In [18]:
df

,text,label,sents,clean_text,words,word_count
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15
...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52


### 词语多样性 word diversity

In [19]:
df['word_diversity'] = df.words.apply(lambda x: len(set(x)))

In [20]:
df

,text,label,sents,clean_text,words,word_count,word_diversity
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15
...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47


### type_token_ratio

In [21]:
df['type_token_ratio'] = df['word_diversity']/df['word_count']

In [22]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000
...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846


### verb_count

In [23]:
def verb_count(words):
    output = ltp.pipeline(words, tasks=["pos"])
    flat_output = [item for sublist in output.pos for item in sublist]
    print(words,':',flat_output)
    verb_count = len([pos for pos in flat_output if pos == 'v'])
    return verb_count

In [24]:
df['verb_count'] = df['words'].apply(verb_count)

['排', '了', '蛮', '久', '味道', '不错', '不太好', '找', '还', '会', '去'] : ['v', 'u', 'd', 'a', 'n', 'a', 'd', 'v', 'd', 'v', 'v']
['咖喱', '很', '好吃', '尤其', '是', '温泉', '蛋', '咖喱', '就是', '麻婆', '茄子', '的', '量', '太少', '不管', '是', '多', '大', '份量', '都', '是', '一样', '的'] : ['n', 'd', 'a', 'd', 'v', 'n', 'n', 'n', 'd', 'n', 'n', 'u', 'n', 'd', 'c', 'v', 'd', 'a', 'n', 'd', 'v', 'a', 'u']
['环境', '很', '整洁', '就是', '一楼', '很', '热', '馄饨', '的', '量', '也', '很', '足'] : ['n', 'd', 'a', 'd', 'm', 'd', 'a', 'n', 'u', 'n', 'd', 'd', 'a']
['很', '有', '特色', '的', '一家', '店', '从', '外面', '电梯', '上去', '直接', '是', '几个', '露天', '的', '蒙古包', '是', '有', '低消', '的', '人均', '貌似', '里面', '环境', '也', '很', '好', '去', '的', '时候', '人不多', '吃', '到', '一半', '有', '歌舞', '表演', '土豆粉', '羊排', '味道', '都', '不错', '本', '以为', '菜量', '很大', '结果', '很小', '人均', '够吃', '啦'] : ['d', 'v', 'n', 'u', 'm', 'n', 'p', 'n', 'd', 'n', 'v', 'a', 'v', 'm', 'n', 'u', 'n', 'v', 'v', 'b', 'u', 'j', 'v', 'n', 'd', 'n', 'd', 'd', 'a', 'v', 'u', 'n', 'n', 'v', 'v', 'm', 'v', 'n', 'v', 'n', 'n', 

['份量', '很足', '四个', '女生', '会', '吃', '到', '撑', '而且', '菜品', '很', '丰富', '食材', '也', '都', '很', '新鲜', '口味', '也', '是', '一级', '棒', '确实', '有', '泰式', '风味', '团', '一下', '能省', '很多', '钱', '值得', '推荐'] : ['n', 'd', 'm', 'n', 'v', 'v', 'v', 'a', 'c', 'n', 'd', 'a', 'n', 'd', 'd', 'd', 'a', 'n', 'd', 'v', 'b', 'a', 'a', 'v', 'b', 'n', 'n', 'm', 'v', 'm', 'n', 'v', 'v']
['酸菜鱼', '特别', '好吃', '鱼肉', '很', '薄', '很', '嫩', '汤', '也', '很', '好喝', '吃', '了', '鱼', '之后', '可以', '再', '涮', '菜', '吃火锅'] : ['n', 'd', 'a', 'n', 'd', 'a', 'd', 'a', 'n', 'd', 'd', 'a', 'v', 'u', 'n', 'n', 'd', 'v', 'd', 'v', 'n', 'v']
['椒', '麻鸡', '好吃', '鸡', '炒', '米粉', '好吃', '酸奶', '好喝', '但是', '最', '好吃', '的', '是', '黄面', '烤肉', '感觉', '是', '上海', '目前为止', '吃', '到', '过', '最', '正宗', '最', '好吃', '的', '黄面', '烤肉'] : ['n', 'n', 'a', 'n', 'n', 'n', 'a', 'n', 'a', 'c', 'd', 'a', 'u', 'v', 'a', 'n', 'v', 'v', 'n', 's', 'n', 't', 'v', 'v', 'u', 'd', 'a', 'd', 'a', 'u', 'a', 'n']
['口感', '很', '好', '很', '适合', '年轻人', '的', '口味', '食材', '也', '很', '新鲜'] : ['n', 'd', 'a',

['与', '价位', '相符', '不好', '不差', '汉堡', '一般', '奶茶', '糖精', '味', '太重'] : ['p', 'n', 'v', 'a', 'd', 'n', 'a', 'n', 'n', 'n', 'd']
['量', '是', '真的', '很足', '味道', '也', '很棒', '部队', '锅', '的', '摆盘', '真心', '喜欢', '看着', '就', '很', '有', '食欲', '问道', '也', '很', '不错', '年糕', '有', '各种', '口味', '各种', '形状', '的', '呢', '萌', '的', '不要', '不要', '的', '以后', '还会', '再', '来', '的', '嗯', '赞赞赞'] : ['n', 'v', 'a', 'd', 'n', 'd', 'd', 'n', 'n', 'u', 'v', 'd', 'v', 'v', 'd', 'd', 'v', 'n', 'v', 'd', 'd', 'a', 'n', 'v', 'r', 'n', 'r', 'n', 'u', 'w', 'p', 'v', 'u', 'i', 'd', 'u', 'n', 'd', 'd', 'd', 'v', 'u', 'w', 'p', 'v']
['口感', '绵绵', '的', '还', '不错', '呢', '抹', '茶', '的', '味道', '也', '挺', '足', '的', '嗯', '不错', '不错'] : ['n', 'z', 'u', 'd', 'a', 'w', 'p', 'n', 'n', 'u', 'n', 'd', 'd', 'a', 'u', 'w', 'p', 'a', 'a']
['蛋糕', '的', '味道', '很棒', '口感', '也', '是', '软软', '的', '简直', '就是', '被', '种草', '了', '正好', '我', '不', '喜欢', '腻', '腻', '的', '味道', '刚好', '感觉', '配', '上', '一杯', '咖啡', '特别', '棒'] : ['n', 'u', 'n', 'd', 'n', 'd', 'v', 'z', 'u', 'd', 'd', 

['无论是', '串串', '还是', '烧烤', '都', '很', '好吃', '店里', '的', '陈设', '很', '有', '情调', '性价比', '高'] : ['c', 'v', 'c', 'v', 'd', 'd', 'a', 'n', 'u', 'n', 'd', 'v', 'n', 'j', 'a']
['外卖', '好', '几次', '点', '的', '都', '是', '这家', '口味', '很', '大众化', '大家', '都', '可以', '接受', '分量', '足', '价格', '也', '很', '便宜', '不想', '出门', '的', '时候', '他们', '家', '是', '个', '不错', '的', '选择', '哦'] : ['n', 'd', 'm', 'm', 'v', 'u', 'd', 'v', 'r', 'n', 'd', 'v', 'r', 'd', 'v', 'v', 'n', 'a', 'n', 'd', 'd', 'a', 'd', 'v', 'u', 'n', 'r', 'n', 'v', 'q', 'a', 'u', 'v', 'u']
['环境', '很', '好', '座位', '设置', '很棒', '无论是', '自己', '一个', '人', '还是', '有', '朋友', '都', '很', '舒适', '二层', '有', '厕所', '很', '方便', '是', '聚餐', '的', '最佳', '选择', '口味', '也', '很棒', '推荐', '的', '是', '土耳其', '烤鸡', '翅', '和', '泡菜', '五花肉', '石锅', '拌', '饭', '分量', '很足', '自己', '一个', '人', '吃', '完全', '够', '了', '我', '还', '加', '了', '果汁', '和', '沙拉', '有点', '浪费', '了'] : ['n', 'd', 'a', 'n', 'v', 'd', 'c', 'r', 'm', 'n', 'c', 'v', 'n', 'd', 'd', 'a', 'm', 'v', 'n', 'd', 'a', 'v', 'v', 'u', 'z', 'v', 'n', 'd'

['两个', '人点', '了', '芝士', '排骨', '和', '小章鱼', '可惜', '同行', '的', '小伙伴', '战斗力', '太低', '了', '否则', '一定', '能', '把', '它', '吃', '完', '味道', '很', '好', '尤其', '是', '饭团', '很香', '然后', '芝士', '排骨', '也', '不错', '排骨', '又', '香', '又', '入味', '嗯', '还有', '章鱼', '还好', '点', '了', '微辣蛮', '辣', '的', '还是', '到', '最后', '我', '就', '把', '它', '当作', '吃', '烤肉', '一样', '包', '着', '生菜', '加', '泡菜', '饭团', '一起', '吃', '简直', '赞', '呀'] : ['m', 'n', 'u', 'n', 'n', 'c', 'a', 'a', 'v', 'u', 'n', 'n', 'd', 'u', 'c', 'd', 'v', 'p', 'r', 'v', 'v', 'n', 'd', 'a', 'd', 'v', 'n', 'd', 'c', 'n', 'n', 'd', 'a', 'n', 'd', 'a', 'd', 'a', 'w', 'p', 'v', 'n', 'd', 'v', 'u', 'a', 'a', 'u', 'd', 'p', 'n', 'd', 'r', 'd', 'p', 'r', 'v', 'v', 'n', 'u', 'v', 'u', 'n', 'v', 'n', 'n', 'd', 'v', 'd', 'a', 'u']
['却是', '充满', '了', '段子', '手', '满满的', '夜店', '风格', '大赞', '吃', '了', '超级', '多', '因为', '是', '真的', '好吃', '特别', '推荐', '他', '的', '那个', '一串', '一串', '的', '肉', '很棒', '块钱', '一位', '的', '自助', '饮料', '也', '很', '划算', '呢', '还', '可以', '一直', '听谦谦', '的', '歌', '喜欢', '如果', '不是', 

...

['酒店', '在', '繁华', '市中心', '交通', '便利', '离', '地铁站', '也', '就', '一', '公里', '左右', '交通', '便利', '位置', '贼', '好', '环境', '真的', '好评', '设计', '的', '很', '用心', '让', '我', '在', '慵懒', '的', '午后', '尽情', '享受', '欧洲', '小镇', '的', '风情'] : ['n', 'p', 'a', 'n', 'l', 'n', 'a', 'p', 'n', 'd', 'd', 'm', 'q', 'm', 'n', 'a', 'n', 'd', 'a', 'n', 'a', 'n', 'v', 'u', 'd', 'a', 'v', 'r', 'p', 'a', 'u', 'n', 't', 'd', 'v', 'n', 's', 'n', 'u', 'n']
['温泉', '样式', '比较', '多', '水质', '一般', '算是', '无功', '无过', '吧', '但是', '房间', '的', '环境', '感觉', '很差', '很', '潮湿', '地板', '上', '还有', '小', '裂缝', '跟', '服务员', '反映', '换房', '也', '是', '不耐烦', '的', '样子', '最后', '干脆', '就', '没人管', '了', '失望'] : ['n', 'n', 'd', 'a', 'n', 'a', 'v', 'i', 'v', 'u', 'c', 'n', 'u', 'n', 'v', 'd', 'd', 'a', 'n', 'n', 'd', 'v', 'a', 'n', 'p', 'n', 'v', 'v', 'd', 'v', 'a', 'u', 'n', 'n', 'd', 'd', 'd', 'v', 'u', 'a']
['和', '同学', '一', '起来', '玩', '的', '哈哈', '离', '景区', '还是', '很近', '的', '也', '很', '方便', '关键', '是', '温泉', '真的', '炒鸡', '好', '啊', '水质', '特别', '好', '泡', '一天', '让', '人', '感觉

['在', '巽', '寮', '湾', '海滩', '旁边', '是', '个', '公园', '地理位置', '超好', '酒店', '比较', '大有', '室内', '室外', '游泳池', '儿童', '游泳', '设备', '很全', '服务态度', '也', '很', '好', '会', '给', '你', '赠送', '免费', '早餐', '而且', '也', '很', '好吃', '总之', '很', '满意', '了'] : ['p', 'n', 's', 'n', 's', 'n', 'n', 'n', 'd', 'v', 'q', 'n', 'n', 'd', 'n', 'd', 'a', 'n', 'l', 'n', 'l', 'n', 'n', 'v', 'n', 'd', 'v', 'd', 'd', 'a', 'v', 'p', 'r', 'v', 'v', 'n', 'c', 'd', 'd', 'a', 'c', 'd', 'v', 'u']
['我', '都', '无心', '吐槽', '了', '人太多', '了', '排队', '就要', '排', '好久', '而且', '根本', '不值', '雪量', '少', '场地', '也', '小', '还', '死', '贵死贵', '坑人', '钱', '而且', '安全性', '我', '觉得', '有', '欠缺', '滑雪', '的话', '我', '觉得', '有', '机会', '还是', '去', '新疆', '或者', '东北', '那边', '滑', '吧'] : ['r', 'd', 'v', 'v', 'u', 'n', 'u', 'v', 'd', 'v', 'm', 'c', 'd', 'd', 'n', 'a', 'n', 'd', 'a', 'd', 'd', 'a', 'v', 'n', 'c', 'n', 'r', 'v', 'v', 'v', 'v', 'u', 'r', 'v', 'v', 'n', 'd', 'v', 'n', 's', 'c', 'n', 'l', 'r', 'v', 'u']
['好不容易', '腾出', '时间', '跟', '家人', '一起', '度假', '放松', '真是', '一个', '好去处', '

['外面', '的', '环境', '算得', '上', '安静', '但', '房间', '环境', '不', '尽如人意', '房间', '的', '布局', '使', '我', '感到', '很', '拥挤', '而且', '那些', '装饰', '很', '俗气', '卫生状况', '很差', '床上', '邹', '巴巴', '的', '有些', '地方', '很', '有', '污垢', '卫生间', '里', '还有', '残留', '的', '头发', '总之', '差评'] : ['n', 'd', 'u', 'n', 'v', 'v', 'a', 'c', 'n', 'n', 'd', 'i', 'n', 'u', 'n', 'v', 'r', 'v', 'd', 'a', 'c', 'r', 'n', 'd', 'a', 'a', 'd', 'n', 'z', 'z', 'u', 'r', 'n', 'd', 'v', 'n', 'n', 'n', 'd', 'v', 'v', 'u', 'n', 'c', 'n']
['环境', '虽然', '还', '可以', '水质', '和', '温度', '也', '马马虎虎', '不过', '服务', '真的', '不怎么样', '价格', '贵', '就', '不谈', '了', '免费', '的', '东西', '没有', '免费', '的', '样子', '像', '糊弄人', '进去', '还有', '好多', '东西', '要', '收费', '有点', '坑爹', '还', '希望', '能', '提高', '服务质量', '毕竟', '泡温泉', '是', '享受', '的', '过程', '服务', '是', '非常', '非常', '重要', '的', '体验'] : ['n', 'c', 'd', 'v', 'n', 'c', 'n', 'd', 'z', 'c', 'v', 'd', 'd', 'n', 'a', 'd', 'd', 'u', 'v', 'u', 'n', 'd', 'v', 'u', 'n', 'v', 'v', 'v', 'v', 'm', 'n', 'v', 'v', 'd', 'a', 'd', 'v', 'v', 'v', 'v', 'd', 'v',

['体验', '很棒', '位于', '景区', '四周', '山水', '环绕', '景色', '优美', '交通', '也', '比较', '方便', '服务', '很', '周到', '很', '贴心', '装饰', '复古', '但', '也', '有', '特色', '设施', '齐全', '温泉', '很', '舒服', '高尔夫', '也', '很', '有趣', '饮食', '也', '很棒', '同时', '也', '实惠', '总体', '给', '人', '一种', '舒适', '的', '感觉', '很', '适合', '独家', '和', '养身'] : ['v', 'd', 'v', 'n', 'n', 'd', 'n', 'v', 'n', 'a', 'n', 'd', 'd', 'a', 'v', 'd', 'a', 'd', 'a', 'n', 'a', 'c', 'd', 'v', 'n', 'n', 'a', 'n', 'd', 'a', 'n', 'd', 'd', 'a', 'n', 'd', 'd', 'c', 'd', 'a', 'n', 'p', 'n', 'm', 'a', 'u', 'n', 'd', 'v', 'b', 'c', 'v']
['总体', '来说', '不怎么', '满意', '体验', '差', '环境', '不怎么', '好', '有点', '吵', '房间', '装饰', '不怎么样', '技师', '手法', '一般', '整体', '收费', '就算', '高', '的', '了', '希望', '可以', '改进', '一下'] : ['n', 'u', 'd', 'v', 'n', 'a', 'n', 'd', 'a', 'd', 'v', 'n', 'v', 'i', 'n', 'n', 'a', 'n', 'v', 'd', 'a', 'u', 'u', 'v', 'v', 'v', 'm']
['和', '朋友', '一起', '去过', '风景', '是', '真心', '不错', '走', '在', '玻璃', '桥', '上', '感觉', '很', '刺激', '最后', '和', '朋友', '互相', '扶', '着', '走', '完', '了', '滑', '草道

['还', '说', '是', '什么', '国际', '花园酒店', '实在', '是', '让', '人', '不敢恭维', '能标', '这个', '价格', '可能', '就是', '因为', '在', '西湖', '旁边', '吧', '首先', '感觉', '床单', '就', '很', '有', '问题', '盖上', '被子', '之后', '也', '不', '舒服', '一', '晚上', '没睡', '好', '卫生间', '里', '也', '不够', '干净', '洗漱', '用具', '也', '不', '齐全', '下次', '不会', '再', '去', '了'] : ['d', 'v', 'v', 'r', 'n', 'n', 'd', 'v', 'v', 'n', 'd', 'v', 'r', 'n', 'v', 'd', 'p', 'p', 'n', 's', 'n', 'd', 'u', 'c', 'v', 'n', 'd', 'd', 'v', 'n', 'v', 'n', 'n', 'd', 'd', 'd', 'a', 'm', 'n', 't', 'd', 'a', 'n', 'n', 'd', 'd', 'd', 'a', 'v', 'n', 'd', 'd', 'a', 'n', 't', 'd', 'd', 'v', 'u']
['水质', '好', '泡起来', '真的', '不错', '周三', '早上', '去', '的', '泡', '了', '一', '早上', '下午', '又', '去', '游乐', '设施', '那边', '挺好玩', '中途', '问', '服务员', '一些', '问题', '的', '时候', '态度', '也', '是', '超级', '好', '很', '亲切', '一家', '温泉', '嘛', '水质', '好', '服务', '好', '怎么', '能', '不', '给', '好评', '呢', '哈哈', '下次', '还来'] : ['n', 'a', 'v', 'd', 'a', 'n', 't', 'n', 't', 'v', 'u', 'v', 'u', 'm', 'n', 't', 'n', 't', 'd', 'v', 'v', 'n', 'r',

In [25]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0
...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9


### sentence_count

In [26]:
df['sentence_count'] = df['sents'].apply(len)

In [27]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2
...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6


### avg_sentence_length

In [28]:
df['avg_sentence_length'] = df['word_count'] / df['sentence_count']

In [29]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,avg_sentence_length
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,11.000000
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,23.000000
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,13.000000
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,51.000000
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,7.500000
...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,10.333333
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,8.333333
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,12.000000
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,8.666667


### causation_ratio

In [30]:
df['causation_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['cause'])[0]) / df['words'].apply(len)

In [31]:
df[df.causation_ratio != 0]

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,avg_sentence_length,causation_ratio
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,51.000000,0.019608
13,第一次来这吃 环境很好 味道也很正宗 团票的话也不是很贵 分量也很足 吃完一本满足 在万达附...,0,[第一次来这吃 环境很好 味道也很正宗 团票的话也不是很贵 分量也很足 吃完一本满足 在万达...,第一次来这吃环境很好味道也很正宗团票的话也不是很贵分量也很足吃完一本满足在万达附近也比较方便...,"[第一次, 来, 这, 吃, 环境, 很, 好, 味道, 也, 很, 正宗, 团票, 的话,...",65,46,0.707692,19,1,65.000000,0.015385
26,先说下排队哈，电话预约的。听闻异常难排，三点准时打电话，其实是按照你几点去拍几号的，有大致时...,0,"[先说下排队哈，电话预约的。, 听闻异常难排，三点准时打电话，其实是按照你几点去拍几号的，有...",先说下排队哈电话预约的听闻异常难排三点准时打电话其实是按照你几点去拍几号的有大致时间段不是先...,"[先说, 下, 排队, 哈, 电话, 预约, 的, 听闻, 异常, 难排, 三点, 准时, ...",113,94,0.831858,26,6,18.833333,0.017699
43,妈妈说想吃自助烧烤，最后选择了赤坂小亭，之前是赤坂亭的忠实粉，但因为妈妈不吃刺生，所以小亭是...,0,[妈妈说想吃自助烧烤，最后选择了赤坂小亭，之前是赤坂亭的忠实粉，但因为妈妈不吃刺生，所以小亭...,妈妈说想吃自助烧烤最后选择了赤坂小亭之前是赤坂亭的忠实粉但因为妈妈不吃刺生所以小亭是个不错的...,"[妈妈, 说, 想, 吃, 自助, 烧烤, 最后, 选择, 了, 赤坂, 小亭, 之前, 是...",54,42,0.777778,16,2,27.000000,0.037037
46,点的红豆薏仁糊 比较稠 虽然味道自然很朴实 但是确实能让人产生一种吃的很健康的错觉，奶盖很好...,0,[点的红豆薏仁糊 比较稠 虽然味道自然很朴实 但是确实能让人产生一种吃的很健康的错觉，奶盖很...,点的红豆薏仁糊比较稠虽然味道自然很朴实但是确实能让人产生一种吃的很健康的错觉奶盖很好喝但是我...,"[点, 的, 红豆, 薏仁, 糊, 比较, 稠, 虽然, 味道, 自然, 很, 朴实, 但是...",41,31,0.756098,9,2,20.500000,0.048780
...,...,...,...,...,...,...,...,...,...,...,...,...
1017,水质好，泡起来真的不错，周三早上去的，泡了一早上，下午又去游乐设施那边，挺好玩。中途问服务员...,1,"[水质好，泡起来真的不错，周三早上去的，泡了一早上，下午又去游乐设施那边，挺好玩。, 中途问...",水质好泡起来真的不错周三早上去的泡了一早上下午又去游乐设施那边挺好玩中途问服务员一些问题的时...,"[水质, 好, 泡起来, 真的, 不错, 周三, 早上, 去, 的, 泡, 了, 一, 早上...",50,43,0.860000,10,4,12.500000,0.020000
1020,新房间霉迹斑斑，让换房也没换成，服务员很懒散，休闲区很差，饮料和水果少的可怜，再也不去了,1,[新房间霉迹斑斑，让换房也没换成，服务员很懒散，休闲区很差，饮料和水果少的可怜，再也不去了],新房间霉迹斑斑让换房也没换成服务员很懒散休闲区很差饮料和水果少的可怜再也不去了,"[新, 房间, 霉迹, 斑斑, 让, 换房, 也, 没, 换成, 服务员, 很, 懒散, 休...",25,24,0.960000,4,1,25.000000,0.040000
1022,抱歉老师我认为一段时间内突然异常的评价如大量差评会对自己的账户甚至隐私造成不良影响所以我不是...,1,[抱歉老师我认为一段时间内突然异常的评价如大量差评会对自己的账户甚至隐私造成不良影响所以我不...,抱歉老师我认为一段时间内突然异常的评价如大量差评会对自己的账户甚至隐私造成不良影响所以我不是...,"[抱歉, 老师, 我, 认为, 一段时间, 内, 突然, 异常, 的, 评价, 如, 大量,...",38,34,0.894737,9,1,38.000000,0.078947
1023,环境算是一般。位置还是挺好的，但周围不是很安静。房间除了空间 其他方面算是完整。服务员比较严...,1,"[环境算是一般。, 位置还是挺好的，但周围不是很安静。, 房间除了空间 其他方面算是完整。,...",环境算是一般位置还是挺好的但周围不是很安静房间除了空间其他方面算是完整服务员比较严肃但速度和...,"[环境, 算是, 一般, 位置, 还是, 挺, 好, 的, 但, 周围, 不是, 很, 安静...",42,37,0.880952,6,6,7.000000,0.023810


### exclusive_ratio

In [32]:
exclusives = read_txtfile('self-made dictionary/exclusive words.txt')
exclusive_count = sum([words.count(w) for w in exclusives])

df['exclusive_ratio'] = df['words'].apply(lambda x: sum(x.count(w) for w in exclusives)) / df['word_count']


In [33]:
df[df.exclusive_ratio != 0]

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,avg_sentence_length,causation_ratio,exclusive_ratio
6,环境和服务超级棒！但可能餐厅比较高档。口味独特。我不是很吃得惯。,0,"[环境和服务超级棒！, 但可能餐厅比较高档。, 口味独特。, 我不是很吃得惯。]",环境和服务超级棒但可能餐厅比较高档口味独特我不是很吃得惯,"[环境, 和, 服务, 超级, 棒, 但, 可能, 餐厅, 比较, 高档, 口味, 独特, ...",17,17,1.000000,3,4,4.250000,0.000000,0.058824
13,第一次来这吃 环境很好 味道也很正宗 团票的话也不是很贵 分量也很足 吃完一本满足 在万达附...,0,[第一次来这吃 环境很好 味道也很正宗 团票的话也不是很贵 分量也很足 吃完一本满足 在万达...,第一次来这吃环境很好味道也很正宗团票的话也不是很贵分量也很足吃完一本满足在万达附近也比较方便...,"[第一次, 来, 这, 吃, 环境, 很, 好, 味道, 也, 很, 正宗, 团票, 的话,...",65,46,0.707692,19,1,65.000000,0.015385,0.015385
14,经常点这家的外卖！红烧肉很好吃！外卖也到的快。就是美食广场里的环境不是特别好。,0,"[经常点这家的外卖！, 红烧肉很好吃！, 外卖也到的快。, 就是美食广场里的环境不是特别好。]",经常点这家的外卖红烧肉很好吃外卖也到的快就是美食广场里的环境不是特别好,"[经常, 点, 这家, 的, 外卖, 红烧肉, 很, 好吃, 外卖, 也, 到, 的, 快,...",22,19,0.863636,3,4,5.500000,0.000000,0.045455
16,牛肉超新鲜！不是冷冻的那种，眼睁睁看着切出来的，哈哈哈,0,"[牛肉超新鲜！, 不是冷冻的那种，眼睁睁看着切出来的，哈哈哈]",牛肉超新鲜不是冷冻的那种眼睁睁看着切出来的哈哈哈,"[牛肉, 超, 新鲜, 不是, 冷冻, 的, 那种, 眼睁睁, 看着, 切出来, 的, 哈哈哈]",12,11,0.916667,3,2,6.000000,0.000000,0.083333
25,"地方还挺好找的呢～有直达电梯！首先,这家店真心是我喜欢的类型，人不是很多，游乐设施有几样，桌...",0,"[地方还挺好找的呢～有直达电梯！, 首先,这家店真心是我喜欢的类型，人不是很多，游乐设施有几...",地方还挺好找的呢有直达电梯首先这家店真心是我喜欢的类型人不是很多游乐设施有几样桌上足球台球还...,"[地方, 还, 挺, 好找, 的, 呢, 有, 直达, 电梯, 首先, 这家, 店, 真心,...",109,80,0.733945,22,9,12.111111,0.000000,0.009174
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010,酒店的地理位置非常的好 但是价格实在是太贵了 最生气的是隔音效果真的非常的差 本人睡眠质量并...,1,[酒店的地理位置非常的好 但是价格实在是太贵了 最生气的是隔音效果真的非常的差 本人睡眠质量...,酒店的地理位置非常的好但是价格实在是太贵了最生气的是隔音效果真的非常的差本人睡眠质量并不高晚...,"[酒店, 的, 地理位置, 非常, 的, 好, 但是, 价格, 实在, 是, 太贵, 了, ...",61,46,0.754098,11,1,61.000000,0.032787,0.016393
1022,抱歉老师我认为一段时间内突然异常的评价如大量差评会对自己的账户甚至隐私造成不良影响所以我不是...,1,[抱歉老师我认为一段时间内突然异常的评价如大量差评会对自己的账户甚至隐私造成不良影响所以我不...,抱歉老师我认为一段时间内突然异常的评价如大量差评会对自己的账户甚至隐私造成不良影响所以我不是...,"[抱歉, 老师, 我, 认为, 一段时间, 内, 突然, 异常, 的, 评价, 如, 大量,...",38,34,0.894737,9,1,38.000000,0.078947,0.026316
1023,环境算是一般。位置还是挺好的，但周围不是很安静。房间除了空间 其他方面算是完整。服务员比较严...,1,"[环境算是一般。, 位置还是挺好的，但周围不是很安静。, 房间除了空间 其他方面算是完整。,...",环境算是一般位置还是挺好的但周围不是很安静房间除了空间其他方面算是完整服务员比较严肃但速度和...,"[环境, 算是, 一般, 位置, 还是, 挺, 好, 的, 但, 周围, 不是, 很, 安静...",42,37,0.880952,6,6,7.000000,0.023810,0.047619
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,12.000000,0.000000,0.041667


## 确信度相关特征

### modality ratio

In [34]:
modal_words = read_txtfile('self-made dictionary/uncertain.txt')
df['modality_ratio'] = df['words'].apply(lambda x: sum([x.count(w) for w in modal_words])) / df['word_count']

In [35]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,avg_sentence_length,causation_ratio,exclusive_ratio,modality_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,11.000000,0.000000,0.000000,0.000000
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,23.000000,0.000000,0.000000,0.000000
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,13.000000,0.000000,0.000000,0.000000
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,51.000000,0.019608,0.000000,0.000000
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,7.500000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,10.333333,0.000000,0.000000,0.000000
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,8.333333,0.000000,0.000000,0.000000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,12.000000,0.000000,0.041667,0.000000
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,8.666667,0.019231,0.019231,0.019231


### certainty ratio

In [36]:
df['certainty_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['certain'])[0]) / df['word_count']

In [37]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,avg_sentence_length,causation_ratio,exclusive_ratio,modality_ratio,certainty_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,11.000000,0.000000,0.000000,0.000000,0.000000
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,23.000000,0.000000,0.000000,0.000000,0.086957
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,13.000000,0.000000,0.000000,0.000000,0.076923
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,51.000000,0.019608,0.000000,0.000000,0.000000
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,7.500000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,10.333333,0.000000,0.000000,0.000000,0.032258
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,8.333333,0.000000,0.000000,0.000000,0.040000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,12.000000,0.000000,0.041667,0.000000,0.000000
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,8.666667,0.019231,0.019231,0.019231,0.057692


## 负面情感相关特征

### negation ratio

In [38]:
df['negation_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['negate'])[0]) / df['word_count']

In [39]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,avg_sentence_length,causation_ratio,exclusive_ratio,modality_ratio,certainty_ratio,negation_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,11.000000,0.000000,0.000000,0.000000,0.000000,0.090909
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,23.000000,0.000000,0.000000,0.000000,0.086957,0.043478
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,13.000000,0.000000,0.000000,0.000000,0.076923,0.000000
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,51.000000,0.019608,0.000000,0.000000,0.000000,0.019608
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,7.500000,0.000000,0.000000,0.000000,0.000000,0.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,10.333333,0.000000,0.000000,0.000000,0.032258,0.032258
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,8.333333,0.000000,0.000000,0.000000,0.040000,0.000000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,12.000000,0.000000,0.041667,0.000000,0.000000,0.166667
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,8.666667,0.019231,0.019231,0.019231,0.057692,0.019231


### negative emotion ratio

In [40]:
df['neg_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['negemo'])[0]) / df['word_count']

In [41]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,avg_sentence_length,causation_ratio,exclusive_ratio,modality_ratio,certainty_ratio,negation_ratio,neg_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,11.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,23.000000,0.000000,0.000000,0.000000,0.086957,0.043478,0.000000
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,13.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,51.000000,0.019608,0.000000,0.000000,0.000000,0.019608,0.000000
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,7.500000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,10.333333,0.000000,0.000000,0.000000,0.032258,0.032258,0.000000
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,8.333333,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,12.000000,0.000000,0.041667,0.000000,0.000000,0.166667,0.125000
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,8.666667,0.019231,0.019231,0.019231,0.057692,0.019231,0.057692


### negative emotions only

In [42]:
df['neg_only_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['anx','anger','sad'])[0]) / df['word_count']

In [43]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,avg_sentence_length,causation_ratio,exclusive_ratio,modality_ratio,certainty_ratio,negation_ratio,neg_ratio,neg_only_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,11.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,23.000000,0.000000,0.000000,0.000000,0.086957,0.043478,0.000000,0.000000
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,13.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,51.000000,0.019608,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,7.500000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,10.333333,0.000000,0.000000,0.000000,0.032258,0.032258,0.000000,0.000000
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,8.333333,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,12.000000,0.000000,0.041667,0.000000,0.000000,0.166667,0.125000,0.000000
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,8.666667,0.019231,0.019231,0.019231,0.057692,0.019231,0.057692,0.019231


### anger & anxiety & sadness

In [44]:
df['anger_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['anger'])[0]) / df['word_count']
df['anxiety_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['anx'])[0]) / df['word_count']
df['sadness_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['sad'])[0]) / df['word_count']

In [45]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,...,causation_ratio,exclusive_ratio,modality_ratio,certainty_ratio,negation_ratio,neg_ratio,neg_only_ratio,anger_ratio,anxiety_ratio,sadness_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,...,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.0,0.0
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,...,0.000000,0.000000,0.000000,0.086957,0.043478,0.000000,0.000000,0.000000,0.0,0.0
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,...,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,...,0.019608,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.0,0.0
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,...,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,...,0.000000,0.000000,0.000000,0.032258,0.032258,0.000000,0.000000,0.000000,0.0,0.0
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,...,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,...,0.000000,0.041667,0.000000,0.000000,0.166667,0.125000,0.000000,0.000000,0.0,0.0
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,...,0.019231,0.019231,0.019231,0.057692,0.019231,0.057692,0.019231,0.019231,0.0,0.0


## 积极情感相关特征

### positive emotion ratio

In [46]:
df['pos_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['posemo'])[0]) / df['word_count']

In [47]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,...,exclusive_ratio,modality_ratio,certainty_ratio,negation_ratio,neg_ratio,neg_only_ratio,anger_ratio,anxiety_ratio,sadness_ratio,pos_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,...,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.0,0.0,0.090909
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,...,0.000000,0.000000,0.086957,0.043478,0.000000,0.000000,0.000000,0.0,0.0,0.086957
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,...,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.076923
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,...,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.0,0.0,0.039216
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,...,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.0,0.0,0.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,...,0.000000,0.000000,0.032258,0.032258,0.000000,0.000000,0.000000,0.0,0.0,0.080645
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,...,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.200000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,...,0.041667,0.000000,0.000000,0.166667,0.125000,0.000000,0.000000,0.0,0.0,0.166667
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,...,0.019231,0.019231,0.057692,0.019231,0.057692,0.019231,0.019231,0.0,0.0,0.019231


### positive 1

### positive 2

## 未指明情感特征

### all emotion ratio

In [48]:
df['allemo_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['posemo', 'negemo'])[0]) / df['word_count']

In [49]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,...,modality_ratio,certainty_ratio,negation_ratio,neg_ratio,neg_only_ratio,anger_ratio,anxiety_ratio,sadness_ratio,pos_ratio,allemo_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,...,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.0,0.0,0.090909,0.090909
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,...,0.000000,0.086957,0.043478,0.000000,0.000000,0.000000,0.0,0.0,0.086957,0.086957
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,...,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.076923,0.076923
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,...,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.0,0.0,0.039216,0.039216
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,...,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.0,0.0,0.133333,0.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,...,0.000000,0.032258,0.032258,0.000000,0.000000,0.000000,0.0,0.0,0.080645,0.080645
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,...,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.200000,0.200000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,...,0.000000,0.000000,0.166667,0.125000,0.000000,0.000000,0.0,0.0,0.166667,0.291667
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,...,0.019231,0.057692,0.019231,0.057692,0.019231,0.019231,0.0,0.0,0.019231,0.076923


### pleasantness ratio

In [50]:
pleasant_words = read_txtfile('self-made dictionary/pleasantness.txt')    
df['pleasant_ratio'] = df['words'].apply(lambda x: sum(x.count(w) for w in pleasant_words))/ df['word_count']

In [51]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,...,certainty_ratio,negation_ratio,neg_ratio,neg_only_ratio,anger_ratio,anxiety_ratio,sadness_ratio,pos_ratio,allemo_ratio,pleasant_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,...,0.000000,0.090909,0.000000,0.000000,0.000000,0.0,0.0,0.090909,0.090909,0.000000
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,...,0.086957,0.043478,0.000000,0.000000,0.000000,0.0,0.0,0.086957,0.086957,0.000000
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,...,0.076923,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.076923,0.076923,0.000000
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,...,0.000000,0.019608,0.000000,0.000000,0.000000,0.0,0.0,0.039216,0.039216,0.019608
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,...,0.000000,0.133333,0.000000,0.000000,0.000000,0.0,0.0,0.133333,0.133333,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,...,0.032258,0.032258,0.000000,0.000000,0.000000,0.0,0.0,0.080645,0.080645,0.064516
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,...,0.040000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.200000,0.200000,0.080000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,...,0.000000,0.166667,0.125000,0.000000,0.000000,0.0,0.0,0.166667,0.291667,0.083333
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,...,0.057692,0.019231,0.057692,0.019231,0.019231,0.0,0.0,0.019231,0.076923,0.000000


## 距离感相关特征

### pronoun ratio 代词比例

In [52]:
df['pronoun_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['pronoun'])[0]) / df['word_count']

In [53]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,...,negation_ratio,neg_ratio,neg_only_ratio,anger_ratio,anxiety_ratio,sadness_ratio,pos_ratio,allemo_ratio,pleasant_ratio,pronoun_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,...,0.090909,0.000000,0.000000,0.000000,0.0,0.0,0.090909,0.090909,0.000000,0.000000
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,...,0.043478,0.000000,0.000000,0.000000,0.0,0.0,0.086957,0.086957,0.000000,0.000000
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.076923,0.076923,0.000000,0.000000
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,...,0.019608,0.000000,0.000000,0.000000,0.0,0.0,0.039216,0.039216,0.019608,0.000000
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,...,0.133333,0.000000,0.000000,0.000000,0.0,0.0,0.133333,0.133333,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,...,0.032258,0.000000,0.000000,0.000000,0.0,0.0,0.080645,0.080645,0.064516,0.000000
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.200000,0.200000,0.080000,0.000000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,...,0.166667,0.125000,0.000000,0.000000,0.0,0.0,0.166667,0.291667,0.083333,0.000000
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,...,0.019231,0.057692,0.019231,0.019231,0.0,0.0,0.019231,0.076923,0.000000,0.057692


### 第一人称单数代词比例

In [54]:
df['fp_sp_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['i'])[0]) / df['word_count']

In [55]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,...,neg_ratio,neg_only_ratio,anger_ratio,anxiety_ratio,sadness_ratio,pos_ratio,allemo_ratio,pleasant_ratio,pronoun_ratio,fp_sp_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,...,0.000000,0.000000,0.000000,0.0,0.0,0.090909,0.090909,0.000000,0.000000,0.0
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,...,0.000000,0.000000,0.000000,0.0,0.0,0.086957,0.086957,0.000000,0.000000,0.0
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,...,0.000000,0.000000,0.000000,0.0,0.0,0.076923,0.076923,0.000000,0.000000,0.0
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,...,0.000000,0.000000,0.000000,0.0,0.0,0.039216,0.039216,0.019608,0.000000,0.0
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,...,0.000000,0.000000,0.000000,0.0,0.0,0.133333,0.133333,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,...,0.000000,0.000000,0.000000,0.0,0.0,0.080645,0.080645,0.064516,0.000000,0.0
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,...,0.000000,0.000000,0.000000,0.0,0.0,0.200000,0.200000,0.080000,0.000000,0.0
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,...,0.125000,0.000000,0.000000,0.0,0.0,0.166667,0.291667,0.083333,0.000000,0.0
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,...,0.057692,0.019231,0.019231,0.0,0.0,0.019231,0.076923,0.000000,0.057692,0.0


### 第一人称复数代词比例

In [56]:
df['fp_pp_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['we'])[0]) / df['word_count']

In [57]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,...,neg_only_ratio,anger_ratio,anxiety_ratio,sadness_ratio,pos_ratio,allemo_ratio,pleasant_ratio,pronoun_ratio,fp_sp_ratio,fp_pp_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,...,0.000000,0.000000,0.0,0.0,0.090909,0.090909,0.000000,0.000000,0.0,0.0
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,...,0.000000,0.000000,0.0,0.0,0.086957,0.086957,0.000000,0.000000,0.0,0.0
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,...,0.000000,0.000000,0.0,0.0,0.076923,0.076923,0.000000,0.000000,0.0,0.0
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,...,0.000000,0.000000,0.0,0.0,0.039216,0.039216,0.019608,0.000000,0.0,0.0
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,...,0.000000,0.000000,0.0,0.0,0.133333,0.133333,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,...,0.000000,0.000000,0.0,0.0,0.080645,0.080645,0.064516,0.000000,0.0,0.0
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,...,0.000000,0.000000,0.0,0.0,0.200000,0.200000,0.080000,0.000000,0.0,0.0
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,...,0.000000,0.000000,0.0,0.0,0.166667,0.291667,0.083333,0.000000,0.0,0.0
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,...,0.019231,0.019231,0.0,0.0,0.019231,0.076923,0.000000,0.057692,0.0,0.0


### 所有第一人称代词比例

In [58]:
df['all_fp_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['i','we'])[0]) / df['word_count']

In [59]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,...,anger_ratio,anxiety_ratio,sadness_ratio,pos_ratio,allemo_ratio,pleasant_ratio,pronoun_ratio,fp_sp_ratio,fp_pp_ratio,all_fp_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,...,0.000000,0.0,0.0,0.090909,0.090909,0.000000,0.000000,0.0,0.0,0.0
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,...,0.000000,0.0,0.0,0.086957,0.086957,0.000000,0.000000,0.0,0.0,0.0
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,...,0.000000,0.0,0.0,0.076923,0.076923,0.000000,0.000000,0.0,0.0,0.0
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,...,0.000000,0.0,0.0,0.039216,0.039216,0.019608,0.000000,0.0,0.0,0.0
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,...,0.000000,0.0,0.0,0.133333,0.133333,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,...,0.000000,0.0,0.0,0.080645,0.080645,0.064516,0.000000,0.0,0.0,0.0
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,...,0.000000,0.0,0.0,0.200000,0.200000,0.080000,0.000000,0.0,0.0,0.0
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,...,0.000000,0.0,0.0,0.166667,0.291667,0.083333,0.000000,0.0,0.0,0.0
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,...,0.019231,0.0,0.0,0.019231,0.076923,0.000000,0.057692,0.0,0.0,0.0


### 所有第二人称代词比例

In [60]:
df['all_sp_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['you'])[0]) / df['word_count']

In [61]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,...,anxiety_ratio,sadness_ratio,pos_ratio,allemo_ratio,pleasant_ratio,pronoun_ratio,fp_sp_ratio,fp_pp_ratio,all_fp_ratio,all_sp_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,...,0.0,0.0,0.090909,0.090909,0.000000,0.000000,0.0,0.0,0.0,0.0
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,...,0.0,0.0,0.086957,0.086957,0.000000,0.000000,0.0,0.0,0.0,0.0
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,...,0.0,0.0,0.076923,0.076923,0.000000,0.000000,0.0,0.0,0.0,0.0
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,...,0.0,0.0,0.039216,0.039216,0.019608,0.000000,0.0,0.0,0.0,0.0
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,...,0.0,0.0,0.133333,0.133333,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,...,0.0,0.0,0.080645,0.080645,0.064516,0.000000,0.0,0.0,0.0,0.0
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,...,0.0,0.0,0.200000,0.200000,0.080000,0.000000,0.0,0.0,0.0,0.0
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,...,0.0,0.0,0.166667,0.291667,0.083333,0.000000,0.0,0.0,0.0,0.0
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,...,0.0,0.0,0.019231,0.076923,0.000000,0.057692,0.0,0.0,0.0,0.0


### 所有第三人称代词比例

In [62]:
df['all_tp_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['shehe'])[0]) / df['word_count']

### passive ratio 被动词汇比例

In [63]:
passive_words = read_txtfile('self-made dictionary/passive.txt')
df['passive_ratio'] = df['words'].apply(lambda x: sum([x.count(w) for w in passive_words])) / df['word_count']

In [64]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,...,pos_ratio,allemo_ratio,pleasant_ratio,pronoun_ratio,fp_sp_ratio,fp_pp_ratio,all_fp_ratio,all_sp_ratio,all_tp_ratio,passive_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,...,0.090909,0.090909,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,...,0.086957,0.086957,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,...,0.076923,0.076923,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,...,0.039216,0.039216,0.019608,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,...,0.133333,0.133333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,...,0.080645,0.080645,0.064516,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,...,0.200000,0.200000,0.080000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,...,0.166667,0.291667,0.083333,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,...,0.019231,0.076923,0.000000,0.057692,0.0,0.0,0.0,0.0,0.0,0.019231


### generalization ratio

In [65]:
general_words = read_txtfile('self-made dictionary/generalization.txt')
df['generalization ratio'] = df['words'].apply(lambda x: sum([x.count(w) for w in general_words])) / df['word_count']

In [66]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,...,allemo_ratio,pleasant_ratio,pronoun_ratio,fp_sp_ratio,fp_pp_ratio,all_fp_ratio,all_sp_ratio,all_tp_ratio,passive_ratio,generalization ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,...,0.090909,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,...,0.086957,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,...,0.076923,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,...,0.039216,0.019608,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,...,0.133333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,...,0.080645,0.064516,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.016129
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,...,0.200000,0.080000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,...,0.291667,0.083333,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,...,0.076923,0.000000,0.057692,0.0,0.0,0.0,0.0,0.0,0.019231,0.000000


## 感知和情景细节相关特征

### perceptual_ratio 感知词+视听触觉词汇

In [67]:
df['perceptual_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['percept'])[0]) / df['word_count']

In [68]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,...,pleasant_ratio,pronoun_ratio,fp_sp_ratio,fp_pp_ratio,all_fp_ratio,all_sp_ratio,all_tp_ratio,passive_ratio,generalization ratio,perceptual_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.090909
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.043478
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.076923
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,...,0.019608,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.058824
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,...,0.064516,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.016129,0.064516
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,...,0.080000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.040000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,...,0.083333,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.041667
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,...,0.000000,0.057692,0.0,0.0,0.0,0.0,0.0,0.019231,0.000000,0.038462


### perceptual_only_ratio 感知觉词汇 -- not yet

In [69]:
#not yet
# def perceptual_only_ratio(text):
#     temporal_words = ['前天', '今天', '明天', '昨天', '时间']
#     words = jieba.lcut(text)
#     temporal_count = sum([words.count(w) for w in temporal_words])
#     return temporal_count / len(words)
    # 感知觉词汇

### seeing_ratio & feeling & hearing

In [70]:
df['seeing_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['see'])[0]) / df['word_count']
df['feeling_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['feel'])[0]) / df['word_count']
df['hearing_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['hear'])[0]) / df['word_count']

### feeling ratio 触觉

### hearing ratio 听觉

### time ratio时间词频

In [71]:
df['time_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['time'])[0]) / df['word_count']
df['space_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['space'])[0]) / df['word_count']
df['timespace_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['time', 'space'])[0]) / df['word_count']

In [72]:
df

,text,label,sents,clean_text,words,word_count,word_diversity,type_token_ratio,verb_count,sentence_count,...,all_tp_ratio,passive_ratio,generalization ratio,perceptual_ratio,seeing_ratio,feeling_ratio,hearing_ratio,time_ratio,space_ratio,timespace_ratio
0,排了蛮久 味道不错 不太好找 还会去,0,[排了蛮久 味道不错 不太好找 还会去],排了蛮久味道不错不太好找还会去,"[排, 了, 蛮, 久, 味道, 不错, 不太好, 找, 还, 会, 去]",11,11,1.000000,4,1,...,0.0,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.090909,0.000000,0.090909
1,咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的,0,[咖喱很好吃，尤其是温泉蛋咖喱，就是麻婆茄子的量太少，不管是多大份，量都是一样的],咖喱很好吃尤其是温泉蛋咖喱就是麻婆茄子的量太少不管是多大份量都是一样的,"[咖喱, 很, 好吃, 尤其, 是, 温泉, 蛋, 咖喱, 就是, 麻婆, 茄子, 的, 量...",23,19,0.826087,3,1,...,0.0,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.086957,0.086957
2,环境很整洁，就是一楼很热，馄饨的量也很足,0,[环境很整洁，就是一楼很热，馄饨的量也很足],环境很整洁就是一楼很热馄饨的量也很足,"[环境, 很, 整洁, 就是, 一楼, 很, 热, 馄饨, 的, 量, 也, 很, 足]",13,11,0.846154,0,1,...,0.0,0.000000,0.000000,0.076923,0.000000,0.076923,0.000000,0.000000,0.076923,0.076923
3,很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环境...,0,[很有特色的一家店～从外面电梯上去直接是几个露天的蒙古包 是有低消的 人均100貌似 里面环...,很有特色的一家店从外面电梯上去直接是几个露天的蒙古包是有低消的人均貌似里面环境也很好去的时候...,"[很, 有, 特色, 的, 一家, 店, 从, 外面, 电梯, 上去, 直接, 是, 几个,...",51,43,0.843137,13,1,...,0.0,0.000000,0.000000,0.058824,0.019608,0.000000,0.000000,0.039216,0.078431,0.117647
4,牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！海蜇头味道也不错,0,"[牛蛙很大一盆 不过还是有点小贵 韭菜蛤蜊很好吃！, 海蜇头味道也不错]",牛蛙很大一盆不过还是有点小贵韭菜蛤蜊很好吃海蜇头味道也不错,"[牛蛙, 很大, 一盆, 不过, 还是, 有点, 小贵, 韭菜, 蛤蜊, 很, 好吃, 海蜇...",15,15,1.000000,0,2,...,0.0,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,景色特别好看，能实现人类和自然的友好接触。安全方面能够实现全面考虑，器材的的种类多，有各种有...,1,"[景色特别好看，能实现人类和自然的友好接触。, 安全方面能够实现全面考虑，器材的的种类多，有...",景色特别好看能实现人类和自然的友好接触安全方面能够实现全面考虑器材的的种类多有各种有趣的预备...,"[景色, 特别, 好看, 能, 实现, 人类, 和, 自然, 的, 友好, 接触, 安全, ...",62,47,0.758065,16,6,...,0.0,0.000000,0.016129,0.064516,0.016129,0.032258,0.000000,0.016129,0.032258,0.048387
1025,水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。景点内还有游乐设施，小孩喜...,1,"[水质好，环境优美，绿化率非常高，和家人一起来的，感觉适合度假旅游。, 景点内还有游乐设施，...",水质好环境优美绿化率非常高和家人一起来的感觉适合度假旅游景点内还有游乐设施小孩喜欢服务也很周到,"[水质, 好, 环境优美, 绿化率, 非常, 高, 和, 家人, 一, 起来, 的, 感觉,...",25,25,1.000000,7,3,...,0.0,0.000000,0.000000,0.040000,0.000000,0.040000,0.000000,0.000000,0.080000,0.080000
1026,环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。总体上不满意，希望店...,1,"[环境不是很好，房间小，不隔音，按摩手法一般，价格比较贵，WiFi不好连。, 总体上不满意，...",环境不是很好房间小不隔音按摩手法一般价格比较贵不好连总体上不满意希望店家能改进,"[环境, 不是, 很, 好, 房间, 小, 不, 隔音, 按摩, 手法, 一般, 价格, 比...",24,23,0.958333,6,2,...,0.0,0.000000,0.000000,0.041667,0.000000,0.000000,0.041667,0.000000,0.166667,0.166667
1027,丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气。...,1,[丝绸之路嘛，就在南山那里，从市里过去也不是很麻烦，所以也跟朋友去过几次了，但是这次真的生气...,丝绸之路嘛就在南山那里从市里过去也不是很麻烦所以也跟朋友去过几次了但是这次真的生气价格比隔壁...,"[丝绸之路, 嘛, 就, 在, 南山, 那里, 从, 市里, 过去, 也, 不是, 很, 麻...",52,47,0.903846,9,6,...,0.0,0.019231,0.000000,0.038462,0.000000,0.000000,0.019231,0.019231,0.076923,0.096154


### space ratio空间

### time+space词频

### prep介词词频

In [86]:
df['prep_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['prep'])[0])/ df['word_count']

In [ ]:
df

### num ratio number词频

In [93]:
df['number_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['number'])[0])/ df['word_count']

In [ ]:
df

### quant词频

In [88]:
df['quant_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['quant'])[0])/ df['word_count']

In [ ]:
df

### modifier ratio adj+adv词频

In [79]:
def modifier(words):
    output = ltp.pipeline(words, tasks=["pos"])
    flat_output = [item for sublist in output.pos for item in sublist]
    modifier_count = len([pos for pos in flat_output if pos == 'a'])
    return modifier_count / len(words)

In [80]:
df['modifier_ratio'] = df['words'].apply(modifier)


### motion ratio motion词频

In [87]:
df['motion_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['motion'])[0])/ df['word_count']

In [ ]:
df

## 认知过程相关特征

### cogproc ratio 认知词频

In [90]:
df['cogproc_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['cogproc'])[0])/ df['word_count']

In [ ]:
df

### insight ratio 洞察词频

In [92]:
    insight_count = count_liwc(words, ['insight'])[0]
df['insight_ratio'] = df['words'].apply(lambda x: count_liwc(x, ['insight'])[0])/ df['word_count']

In [102]:
df[df.exclusive_ratio != 0].exclusive_ratio

6       0.058824
13      0.015385
14      0.045455
16      0.083333
25      0.009174
          ...   
1010    0.016393
1022    0.026316
1023    0.047619
1026    0.041667
1027    0.019231
Name: exclusive_ratio, Length: 136, dtype: float64

In [104]:
df.iloc[:5,7:25]

,type_token_ratio,verb_count,sentence_count,avg_sentence_length,causation_ratio,exclusive_ratio,modality_ratio,certainty_ratio,negation_ratio,neg_ratio,neg_only_ratio,anger_ratio,anxiety_ratio,sadness_ratio,pos_ratio,allemo_ratio,pleasant_ratio,pronoun_ratio
0,1.000000,4,1,11.0,0.000000,0.0,0.0,0.000000,0.090909,0.0,0.0,0.0,0.0,0.0,0.090909,0.090909,0.000000,0.0
1,0.826087,3,1,23.0,0.000000,0.0,0.0,0.086957,0.043478,0.0,0.0,0.0,0.0,0.0,0.086957,0.086957,0.000000,0.0
2,0.846154,0,1,13.0,0.000000,0.0,0.0,0.076923,0.000000,0.0,0.0,0.0,0.0,0.0,0.076923,0.076923,0.000000,0.0
3,0.843137,13,1,51.0,0.019608,0.0,0.0,0.000000,0.019608,0.0,0.0,0.0,0.0,0.0,0.039216,0.039216,0.019608,0.0
4,1.000000,0,2,7.5,0.000000,0.0,0.0,0.000000,0.133333,0.0,0.0,0.0,0.0,0.0,0.133333,0.133333,0.000000,0.0


In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1029 entries, 0 to 1028
Data columns (total 46 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   text                  1029 non-null   object 
 1   label                 1029 non-null   int64  
 2   sents                 1029 non-null   object 
 3   clean_text            1029 non-null   object 
 4   words                 1029 non-null   object 
 5   word_count            1029 non-null   int64  
 6   word_diversity        1029 non-null   int64  
 7   type_token_ratio      1029 non-null   float64
 8   verb_count            1029 non-null   int64  
 9   sentence_count        1029 non-null   int64  
 10  avg_sentence_length   1029 non-null   float64
 11  causation_ratio       1029 non-null   float64
 12  exclusive_ratio       1029 non-null   float64
 13  modality_ratio        1029 non-null   float64
 14  certainty_ratio       1029 non-null   float64
 15  negation_ratio       

## 保存特征

In [208]:
columns_to_write = df.columns[5:]
columns_to_write = columns_to_write.append(pd.Index(['label']))
df.to_excel('features_v1.xlsx',columns=columns_to_write)

# 实验

In [28]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost as xgb
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import json
from hyperopt import space_eval
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import os
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


In [ ]:
features = pd.read_excel('features_v1.xlsx', index_col=0)
labels = features.iloc[:, -1]
features = features.iloc[:,:-1]

## 纯语言特征 -- 0.86

#### 标准化

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# 创建MinMaxScaler对象
scaler_minmax = MinMaxScaler()

# 选择整数类型的列
int_columns = X_train.select_dtypes(include=['int', 'int64']).columns
# 假设 avg_sentence_length 是整数或者类似整数需要标准化的列
int_columns = int_columns.append(pd.Index(['avg_sentence_length']))

# 对训练集的整数类型列进行Min-Max标准化
X_train[int_columns] = scaler_minmax.fit_transform(X_train[int_columns])
# 对测试集的整数类型列进行Min-Max标准化，注意此处只有transform,即使用训练集进行fit，并按照训练集尺度进行transform测试集
X_test[int_columns] = scaler_minmax.transform(X_test[int_columns])

### xgboost

In [218]:


X = df.iloc[:, 5:]  # 特征值
y = df.iloc[:, 1]   # 标签

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建XGBoost模型
xgb_model = xgb.XGBClassifier()

# 设置要测试的参数网格
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'gamma': [0, 0.1, 0.3]
}

# 创建GridSearchCV对象，使用5折交叉验证
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy')

# 运行网格搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数
print(f"Best parameters found: {grid_search.best_params_}")

# 使用最佳参数的模型进行预测
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# 输出分类报告
print(classification_report(y_test, y_pred))

Best parameters found: {'gamma': 0.3, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}
Accuracy: 0.8640776699029126
              precision    recall  f1-score   support

           0       0.92      0.79      0.85       102
           1       0.82      0.93      0.87       104

    accuracy                           0.86       206
   macro avg       0.87      0.86      0.86       206
weighted avg       0.87      0.86      0.86       206



### SVM

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
import pandas as pd

# 假设 df 是你的 DataFrame
X = features  # 特征值
y = labels   # 标签

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
 
# 创建MinMaxScaler对象
scaler_minmax = MinMaxScaler()

# 选择整数类型的列
int_columns = X_train.select_dtypes(include=['int', 'int64']).columns
# 假设 avg_sentence_length 是整数或者类似整数需要标准化的列
int_columns = int_columns.append(pd.Index(['avg_sentence_length']))

# 对训练集的整数类型列进行Min-Max标准化
X_train[int_columns] = scaler_minmax.fit_transform(X_train[int_columns])
# 对测试集的整数类型列进行Min-Max标准化，注意此处只有transform,即使用训练集进行fit，并按照训练集尺度进行transform测试集
X_test[int_columns] = scaler_minmax.transform(X_test[int_columns])

In [82]:
# 交叉验证
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
# SVM
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, cross_val_score
from bayes_opt import BayesianOptimization

svm = SVC()

def svm_cv(C):
    scores = cross_val_score(
        SVC(kernel='linear',
            C=C
        ),
        X_train, y_train, scoring='accuracy', cv=kfold 
    ).mean()
    return scores

svm_bo = BayesianOptimization(
    svm_cv,
    {'C': (1e-04, 1e04),}
)

svm_bo.maximize()
print(svm_bo.res)


|   iter    |  target   |     C     |
-------------------------------------
| 1         | 0.8226    | 766.0     |
| 2         | 0.8043    | 2.501e+03 |
| 3         | 0.8165    | 5.531e+03 |
| 4         | 0.8213    | 3.227e+03 |
| 5         | 0.8202    | 7.178e+03 |
| 6         | 0.8287    | 764.5     |
| 7         | 0.8287    | 760.3     |
| 8         | 0.8274    | 754.9     |
| 9         | 0.8335    | 749.7     |
| 10        | 0.8311    | 743.9     |
| 11        | 0.8287    | 739.0     |
| 12        | 0.8335    | 729.7     |
| 13        | 0.8286    | 725.0     |
| 14        | 0.8262    | 714.4     |
| 15        | 0.8323    | 733.4     |


## bertclassification

## 有训练的bert模型 -- 0.92

In [ ]:
classification report:{
    "0": {
        "precision": 0.8771929824561403,
        "recall": 0.9803921568627451,
        "f1-score": 0.9259259259259259,
        "support": 102.0
    },
    "1": {
        "precision": 0.9782608695652174,
        "recall": 0.8653846153846154,
        "f1-score": 0.9183673469387755,
        "support": 104.0
    },
    "accuracy": 0.9223300970873787,
    "macro avg": {
        "precision": 0.9277269260106789,
        "recall": 0.9228883861236803,
        "f1-score": 0.9221466364323507,
        "support": 206.0
    },
    "weighted avg": {
        "precision": 0.928217546821888,
        "recall": 0.9223300970873787,
        "f1-score": 0.9221099443013452,
        "support": 206.0
    }
}
Accuracy: 0.9223300970873787
F1 Score: 0.9221099443013452
Recall: 0.9223300970873787
Precision: 0.928217546821888


## 未训练的Bert模型

## 未经过微调得到的Bert词向量&语言特征 -- 0.89

In [73]:
vec = pd.read_excel('bert_vector/bert_vec.xlsx', index_col=0)

In [ ]:
vec

In [ ]:
features

In [ ]:
features.info()

In [ ]:
vec.describe()

### 直接融合 -- 0.90

#### 特征标准化

In [50]:
def prep(features,vec,labels):
    # 拆分数据集
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # 创建MinMaxScaler对象
    scaler_minmax = MinMaxScaler()

    # 选择整数类型的列
    int_columns = X_train.select_dtypes(include=['int', 'int64']).columns
    # 假设 avg_sentence_length 是整数或者类似整数需要标准化的列
    int_columns = int_columns.append(pd.Index(['avg_sentence_length']))

    # 对训练集的整数类型列进行Min-Max标准化
    X_train[int_columns] = scaler_minmax.fit_transform(X_train[int_columns])
    # 对测试集的整数类型列进行Min-Max标准化，注意此处只有transform,即使用训练集进行fit，并按照训练集尺度进行transform测试集
    X_test[int_columns] = scaler_minmax.transform(X_test[int_columns])

    # 对BERT向量进行Min-Max标准化
    # 先对训练集进行fit_transform
    vec_train = scaler_minmax.fit_transform(vec.loc[X_train.index, :])
    # 然后对测试集应用相同的变换
    vec_test = scaler_minmax.transform(vec.loc[X_test.index, :])

    # 合并训练集和测试集的特征
    X_train_combined = np.hstack((X_train, vec_train))
    X_test_combined = np.hstack((X_test, vec_test))
    
    return X_train_combined,X_test_combined, y_train, y_test

#### 找到最佳参数

In [234]:
# 定义目标函数
def objective(space):
    clf = xgb.XGBClassifier(
        n_estimators=int(space['n_estimators']), 
        max_depth=int(space['max_depth']),
        min_child_weight=space['min_child_weight'],
        gamma=space['gamma'],
        learning_rate=space['learning_rate'],
        use_label_encoder=False,
        eval_metric='mlogloss'
    )
    
    # 使用交叉验证评估模型
    score = cross_val_score(clf, X_train_combined, y_train, cv=3, scoring='accuracy').mean()
    
    # 由于我们想要最大化准确率，因此返回负准确率作为损失
    return {'loss': -score, 'status': STATUS_OK}

# 定义超参数空间
space = {
    'n_estimators': hp.quniform('n_estimators', 90, 110, 1),
    'max_depth': hp.quniform('max_depth', 5, 7, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
    'gamma': hp.quniform('gamma', 0, 0.1, 0.01),
    'learning_rate': hp.quniform('learning_rate', 0.25, 0.35, 0.01)
}

# 运行 hyperopt
trials = Trials()
best_hyperparams = fmin(fn=objective,
                        space=space,
                        algo=tpe.suggest,
                        max_evals=100,
                        trials=trials)

print("The best hyperparameters are: ", "\n")
print(best_hyperparams)

100%|█████████████████████████████████████████████| 100/100 [12:31<00:00,  7.52s/trial, best loss: -0.8918734793187347]
The best hyperparameters are:  

{'gamma': 0.04, 'learning_rate': 0.32, 'max_depth': 6.0, 'min_child_weight': 1.0, 'n_estimators': 91.0}


#### 模型再训练

In [ ]:
# 将超参数转化为具体的数值
best_hyperparams = space_eval(space, best_hyperparams)
best_hyperparams['max_depth'] = int(best_hyperparams['max_depth'])
best_hyperparams['n_estimators'] = int(best_hyperparams['n_estimators'])
best_hyperparams['min_child_weight'] = int(best_hyperparams['min_child_weight'])

# 使用最佳超参数训练模型
best_model = xgb.XGBClassifier(**best_hyperparams)
best_model.fit(X_train_combined, y_train)

# 在单独的测试集上评估模型性能
predictions = best_model.predict(X_test_combined)
accuracy = accuracy_score(y_test, predictions)
report = {'accuracy': accuracy}

# 保存模型
best_model.save_model('model params/mixfeatures/768_41_model.json')

# 保存模型性能指标，这里需要您根据实际情况定义 report 变量
with open('model params/mixfeatures/768_41_hyperparams.txt', 'w') as f:
    f.write("Best model performance:\n")
    f.write(json.dumps(report, indent=4) + "\n")
    f.write(f'accuracy:{accuracy}\n')

In [232]:

# 初始化XGBoost模型
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# 训练模型
xgb_model.fit(X_train_combined, y_train)

# 预测
predictions = xgb_model.predict(X_test_combined)

# 性能评估
print(classification_report(y_test, predictions))
# 输出模型参数
print("Model parameters:")
print(xgb_model.get_params())

              precision    recall  f1-score   support

           0       0.93      0.87      0.90       102
           1       0.88      0.93      0.91       104

    accuracy                           0.90       206
   macro avg       0.90      0.90      0.90       206
weighted avg       0.90      0.90      0.90       206

Model parameters:
{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'mlogloss', 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators'

In [ ]:

# 模型微调 - 使用网格搜索来寻找最优参数
param_grid = {
    'n_estimators': [50, 100,150,200],
    'max_depth': [3,4, 5], 
    'min_child_weight': [3, 5],
    'gamma':[i/10.0 for i in range(3,5)],
    'learning_rate': [0.01, 0.05, 0.1]
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1)

grid_search.fit(X_train_combined, y_train)

# 最佳参数和模型
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# 使用最佳模型进行预测
best_predictions = best_model.predict(X_test_combined)


In [ ]:
# 最佳模型的性能评估
print("Best model performance:")
print('best param:',best_params)
print(classification_report(y_test, best_predictions))

# 保存模型
best_model.save_model('best_xgboost_model_mixfeatures.json')
with open('model/xgboost_mixfeatures_perf.txt','w') as f:
    f.write(f"best model performance:")
    f.write(json.dumps(report, indent=4) + "\n")  # JSON format for readability


In [ ]:
# 特征重要性：评估特征的重要性
feature_importances = best_model.feature_importances_
# 打印特征重要性
print("特征重要性：", feature_importances)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 假设 feature_importances 是一个包含特征重要性的数组
# 并且我们有一个特征名称列表，与 feature_importances 中的数值对应
feature_names = ['Feature1', 'Feature2', 'Feature3', ...]

# 对特征重要性进行排序
sorted_idx = np.argsort(feature_importances)
sorted_importances = feature_importances[sorted_idx]
sorted_features = [feature_names[i] for i in sorted_idx]

# 创建条形图
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_importances)), sorted_importances, color='skyblue')
plt.yticks(range(len(sorted_importances)), sorted_features)

# 添加图形标题和轴标签
plt.title('Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')

# 展示图形
plt.show()

#### 模型再训练（整个数据集）

In [ ]:
X_full = full_data.drop('target', axis=1)
y_full = full_data['target']

# 特征标准化
scaler = StandardScaler()
X_full_scaled = scaler.fit_transform(X_full)

# 创建DMatrix对象，XGBoost优化的数据结构
dtrain_full = xgb.DMatrix(X_full_scaled, label=y_full)

# 设置最优参数
params = {'max_depth': 3, 'eta': 0.1, 'objective': 'binary:logistic', 'eval_metric': 'auc'}

# 训练模型
final_model = xgb.train(params, dtrain_full, num_boost_round=100)

# 保存最终模型
final_model.save_model('final_model.json')

#### 模型复现

In [ ]:

# # 加载模型
# bst = xgb.Booster()
# bst.load_model('best_xgboost_model_mixfeatures.json')

# # 假设你有一些要预测的新数据存储在new_data.csv文件中
# # 你需要先将新数据加载到pandas DataFrame中
# new_data = pd.read_csv('path_to_new_data.csv')

# # 然后将新数据转换为DMatrix，因为XGBoost使用DMatrix格式
# # 这里假设要预测的特征已经准备好了，并且不需要额外的处理
# dnew = xgb.DMatrix(new_data)

# # 使用加载的模型进行预测
# predictions = bst.predict(dnew)

# # 输出预测结果
# print(predictions)

### PCA降维 -- 0.89

In [69]:
# 选择目标维度
pca = PCA(n_components=41)
vec_reduced = pd.DataFrame(pca.fit_transform(vec))
X_train_combined,X_test_combined, y_train, y_test = prep(features,vec_reduced,labels) 


In [76]:
# 初始化XGBoost模型
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# 训练模型
xgb_model.fit(X_train_combined, y_train)

# 预测
predictions = xgb_model.predict(X_test_combined)

# 性能评估
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       102
           1       1.00      1.00      1.00       104

    accuracy                           1.00       206
   macro avg       1.00      1.00      1.00       206
weighted avg       1.00      1.00      1.00       206



In [264]:
# 定义目标函数
def objective(space):
    clf = xgb.XGBClassifier(
        n_estimators=int(space['n_estimators']), 
        max_depth=int(space['max_depth']),
        min_child_weight=space['min_child_weight'],
        gamma=space['gamma'],
        learning_rate=space['learning_rate'],
        use_label_encoder=False,
        eval_metric='mlogloss'
    )
    
    # 使用交叉验证评估模型
    score = cross_val_score(clf, X_train_combined, y_train, cv=3, scoring='accuracy').mean()
    
    # 由于我们想要最大化准确率，因此返回负准确率作为损失
    return {'loss': -score, 'status': STATUS_OK}

# 定义超参数空间
space = {
    'n_estimators': hp.quniform('n_estimators', 90, 110, 1),
    'max_depth': hp.quniform('max_depth', 4, 6, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 3, 1),
    'gamma': hp.quniform('gamma', 0, 0.1, 0.01),
    'learning_rate': hp.quniform('learning_rate', 0.25, 0.35, 0.01)
}

# 运行 hyperopt
trials = Trials()
best_hyperparams = fmin(fn=objective,
                        space=space,
                        algo=tpe.suggest,
                        max_evals=100,
                        trials=trials)
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)

100%|█████████████████████████████████████████████| 100/100 [01:34<00:00,  1.06trial/s, best loss: -0.8930944481309444]
The best hyperparameters are:  

{'gamma': 0.06, 'learning_rate': 0.32, 'max_depth': 5.0, 'min_child_weight': 2.0, 'n_estimators': 107.0}


In [259]:
feature_importances = xgb_model.feature_importances_

In [260]:
feature_importances

array([0.0206185 , 0.22635098, 0.003481  , 0.02200999, 0.01256937,
       0.00306701, 0.00110126, 0.        , 0.        , 0.0034444 ,
       0.02097908, 0.01388194, 0.00636771, 0.03154901, 0.        ,
       0.        , 0.00993033, 0.00109424, 0.01204907, 0.00118982,
       0.        , 0.        , 0.02727222, 0.        , 0.        ,
       0.00419594, 0.03273294, 0.00658662, 0.00953535, 0.00387187,
       0.        , 0.0276068 , 0.01805881, 0.00926574, 0.00858909,
       0.02532818, 0.00986187, 0.01319562, 0.01639106, 0.01623956,
       0.00793523, 0.03892563, 0.01520296, 0.00591585, 0.01301414,
       0.01205406, 0.01253488, 0.00697078, 0.0163522 , 0.01087549,
       0.00546104, 0.0051979 , 0.02184057, 0.01353753, 0.00677409,
       0.00794461, 0.00831355, 0.00370285, 0.00830899, 0.0085913 ,
       0.00655764, 0.00517146, 0.01306883, 0.00969027, 0.00491668,
       0.01159961, 0.00778434, 0.00216093, 0.00552977, 0.00340975,
       0.00651667, 0.00635357, 0.00684151, 0.0112858 , 0.01092

## 经过微调得到的BERT词向量

虚假评论文本检测是一个复杂的文本分类任务，其目的是区分真实评论和虚假（或者说是生成的）评论。在这种情况下，BERT模型不仅仅要捕捉基本的语言特征，还需要理解评论文本中的微妙线索，这可能与训练BERT的原始语料库中的数据特性有所不同。

鉴于本数据集是线上餐厅点评的评论文本，它属于一个特定的应用领域。对于这样的特定领域数据集，尽管我的数据集规模相对较小（1000条数据），微调仍然是推荐的，因为这可以帮助BERT更好地适应特定领域的语言表达方式，从而在特征提取时能捕捉到更加细致的语义差异，这对于虚假评论检测是十分重要的。

进行微调时，需要注意以下几点：

鉴于数据集较小，为防止过拟合，可以采取一些策略，比如使用dropout、权重衰减（L2正则化）、早停（early stopping）等技术。
采用交叉验证来更好地评估模型的性能和泛化能力。
尝试使用不同的学习率和批次大小，并监控验证集的损失，以找到最佳的超参数设置。
如果资源允许，可以尝试使用已经在类似领域数据上预训练过的BERT模型，如餐厅评论或相关领域的BERT模型。

### 未降维直接融合心理学特征

In [51]:
def get_file_paths(directory):
    file_paths = []  # List to store file paths
    for root, directories, files in os.walk(directory):
        for filename in files:
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
    return file_paths


In [52]:
def general_train(X_train_combined,X_test_combined,y_train,y_test):
    # 初始化XGBoost模型
    xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

    # 训练模型
    xgb_model.fit(X_train_combined, y_train)

    # 预测
    predictions = xgb_model.predict(X_test_combined)
    print(classification_report(y_test, predictions))
    # 性能评估
#     return classification_report(y_test, predictions)

In [63]:
directory_path = 'embeddings'
file_paths = get_file_paths(directory_path)
for path in file_paths:
    vec = pd.read_excel(path, index_col=0,header=0)
    X_train_combined,X_test_combined, y_train, y_test = prep(features,vec,labels) 
    rep = general_train(X_train_combined,X_test_combined, y_train, y_test)
    print(path)
    print(rep)

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       102
           1       0.96      0.89      0.93       104

    accuracy                           0.93       206
   macro avg       0.93      0.93      0.93       206
weighted avg       0.93      0.93      0.93       206

embeddings\cls_embs.xlsx
None
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       102
           1       0.92      0.93      0.92       104

    accuracy                           0.92       206
   macro avg       0.92      0.92      0.92       206
weighted avg       0.92      0.92      0.92       206

embeddings\max_embs.xlsx
None
              precision    recall  f1-score   support

           0       0.88      0.95      0.92       102
           1       0.95      0.88      0.91       104

    accuracy                           0.91       206
   macro avg       0.91      0.91      0.91       206
weighted avg  

### 单独 CLS MAX MEAN

In [300]:
for path in file_paths:
    vec = pd.read_excel(path, index_col=0,header=0)
    X_train, X_test, y_train, y_test = train_test_split(vec, labels, test_size=0.2, random_state=42)
    print(path)
    general_train(X_train, X_test, y_train, y_test)

embeddings\cls_embs.xlsx
              precision    recall  f1-score   support

           0       0.89      0.97      0.93       102
           1       0.97      0.88      0.92       104

    accuracy                           0.93       206
   macro avg       0.93      0.93      0.93       206
weighted avg       0.93      0.93      0.93       206

embeddings\max_embs.xlsx
              precision    recall  f1-score   support

           0       0.90      0.92      0.91       102
           1       0.92      0.90      0.91       104

    accuracy                           0.91       206
   macro avg       0.91      0.91      0.91       206
weighted avg       0.91      0.91      0.91       206

embeddings\mean_embs.xlsx
              precision    recall  f1-score   support

           0       0.87      0.95      0.91       102
           1       0.95      0.87      0.90       104

    accuracy                           0.91       206
   macro avg       0.91      0.91      0.91       20

### 降维后融合语言特征

In [301]:
# 选择目标维度
pca = PCA(n_components=41)
vec_reduced = pd.DataFrame(pca.fit_transform(vec))
X_train_combined,X_test_combined, y_train, y_test = prep(features,vec_reduced,labels) 

for path in file_paths:
    pca = PCA(n_components=41)
    vec_reduced = pd.DataFrame(pca.fit_transform(vec))
    X_train_combined,X_test_combined, y_train, y_test = prep(features,vec_reduced,labels) 
    print(path)
    general_train(X_train, X_test, y_train, y_test)

embeddings\cls_embs.xlsx
              precision    recall  f1-score   support

           0       0.87      0.95      0.91       102
           1       0.95      0.87      0.90       104

    accuracy                           0.91       206
   macro avg       0.91      0.91      0.91       206
weighted avg       0.91      0.91      0.91       206

embeddings\max_embs.xlsx
              precision    recall  f1-score   support

           0       0.87      0.95      0.91       102
           1       0.95      0.87      0.90       104

    accuracy                           0.91       206
   macro avg       0.91      0.91      0.91       206
weighted avg       0.91      0.91      0.91       206

embeddings\mean_embs.xlsx
              precision    recall  f1-score   support

           0       0.87      0.95      0.91       102
           1       0.95      0.87      0.90       104

    accuracy                           0.91       206
   macro avg       0.91      0.91      0.91       20

| Feature Set                       | Performance |
|-----------------------------------|-------------|
| Psycholinguistic Features (41-dim) | 0.86        |
| Untuned Bert - Max Pooling Vectors | 0.90        |
| Tuned Bert - CLS Vectors           | 0.91        |
| Tuned Bert - Max Pooling Vectors   | 0.91        |
| Untuned Bert - Mean Pooling Vectors| 0.91        |
| Composite Features (Psycholinguistic + X) |           |
| Untuned Bert - Max Pooling Vectors | 0.90        |
| Untuned Bert - Max Pooling Vectors-- reduced（41-dim） | 0.89        |
| Tuned Bert - CLS Vectors           | 0.93        |
| Tuned Bert - Max Pooling Vectors   | 0.92        |
| Untuned Bert - Mean Pooling Vectors| 0.91        |
| Direct Classification with Tuned BERT Model | 0.92    |

| 特征集合                       | 性能指标 |
|--------------------------------|----------|
| 单独类别的特征          |      |
| 心理语言学特征（41维）          | 0.86     |
| 未经微调的Bert-最大池化向量 （768维）    | 0.90     |
| 经过微调的Bert-CLS向量 （768维）       | 0.96     |
| 经过微调的Bert-最大池化向量 （768维）   | 0.96     |
| 未经微调的Bert-平均池化向量 （768维）   | 0.96     |
| 融合特征（心理学特征+X） -- （41+768维）      |          |
| 未经微调的Bert-最大池化向量    | 0.90     |
| 经过微调的Bert-CLS向量         | 0.93     |
| 经过微调的Bert-最大池化向量    | 0.92     |
| 未经微调的Bert-平均池化向量    | 0.91     |
| 融合特征（心理学特征+X）降维后 -- （41+41维）      |          |
| 未经微调的Bert-最大池化向量    | 0.89     |
| 经过微调的Bert-CLS向量         | 0.93     |
| 经过微调的Bert-最大池化向量    | 0.93     |
| 经过微调的Bert-平均池化向量    | 0.94     |

In [62]:
def optimize_hyperparameters(X_train, y_train):
    # 定义目标函数
    def objective(space):
        clf = xgb.XGBClassifier(
            n_estimators=int(space['n_estimators']), 
            max_depth=int(space['max_depth']),
            min_child_weight=int(space['min_child_weight']),
            gamma=space['gamma'],
            learning_rate=space['learning_rate'],
            use_label_encoder=False,
            eval_metric='mlogloss'
        )
        
        # 使用交叉验证评估模型
        score = cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy').mean()
        
        # 由于我们想要最大化准确率，因此返回负准确率作为损失
        return {'loss': -score, 'status': STATUS_OK}

    # 定义超参数空间
    space = {
        'n_estimators': hp.quniform('n_estimators', 90, 110, 1),
        'max_depth': hp.quniform('max_depth', 5, 7, 1),
        'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
        'gamma': hp.quniform('gamma', 0, 0.1, 0.01),
        'learning_rate': hp.quniform('learning_rate', 0.25, 0.35, 0.01)
    }

    # 运行 hyperopt
    trials = Trials()
    best_hyperparams = fmin(fn=objective,
                            space=space,
                            algo=tpe.suggest,
                            max_evals=100,
                            trials=trials)
    
    # 取得最优参数及其对应的性能
    best_score = -min(trials.losses())
    
    return best_hyperparams, best_score



In [ ]:
best_params_list = []
best_perf_list = [] 
for path in file_paths:
    pca = PCA(n_components=41)
    vec_reduced = pd.DataFrame(pca.fit_transform(vec))
    X_train_combined,X_test_combined, y_train, y_test = prep(features,vec_reduced,labels) 
    
    # 使用函数
    best_params, best_perf = optimize_hyperparameters(X_train_combined, y_train)
    best_params_list.append(best_params)
    best_perf_list.append(best_perf)
    print("The best hyperparameters are:", best_params)
    print("Best performance:", best_perf)
    print(path)

In [59]:
def evaluate_model(best_hyperparams, X_train, y_train, X_test, y_test):
    # 使用最优超参数创建模型
    clf = xgb.XGBClassifier(
        n_estimators=int(best_hyperparams['n_estimators']),
        max_depth=int(best_hyperparams['max_depth']),
        min_child_weight=int(best_hyperparams['min_child_weight']),
        gamma=best_hyperparams['gamma'],
        learning_rate=best_hyperparams['learning_rate'],
        use_label_encoder=False,
        eval_metric='mlogloss'
    )
    
    # 在完整的训练数据上训练模型
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # 在测试集上评估模型
    test_accuracy = clf.score(X_test, y_test)
    report = classification_report(y_test, y_pred)

#     # 打印分类报告
#     print("Test Accuracy:", test_accuracy)
#     print("\nClassification Report:\n", report)
    
    return test_accuracy, report

In [60]:
for path in file_paths:
    pca = PCA(n_components=41)
    vec_reduced = pd.DataFrame(pca.fit_transform(vec))
    X_train_combined,X_test_combined, y_train, y_test = prep(features,vec_reduced,labels) 
    
    # 调用优化函数获取最优超参数
    best_params, best_perf = optimize_hyperparameters(X_train_combined, y_train)

    # 使用测试集评估最优模型
    test_accuracy, report = evaluate_model(best_params, X_train_combined, y_train, X_test_combined, y_test)
#     print(f'{path}:/n')
#     print(report)
    # 打印分类报告
    print("Test Accuracy:", test_accuracy)
    print("\nClassification Report:\n", report)

100%|██████████████████████████████████████████████| 100/100 [00:38<00:00,  2.62trial/s, best loss: -0.969626188896262]
Test Accuracy: 0.9320388349514563

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.97      0.93       102
           1       0.97      0.89      0.93       104

    accuracy                           0.93       206
   macro avg       0.93      0.93      0.93       206
weighted avg       0.93      0.93      0.93       206

100%|█████████████████████████████████████████████| 100/100 [00:44<00:00,  2.23trial/s, best loss: -0.9708515815085158]
Test Accuracy: 0.9320388349514563

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.95      0.93       102
           1       0.95      0.91      0.93       104

    accuracy                           0.93       206
   macro avg       0.93      0.93      0.93       206
weighted avg       0.93      0.93      0.93

#### 真实评论有524条，虚假评论有505条
类别分布相对均衡，因为两个类别的样本数量非常接近，没有显著的不平衡。

In [13]:
labels[labels==0].count()

524

In [14]:
labels[labels==1].count()

505

## 结果解读
因为我们希望尽可能的识别出虚假评论，且避免将真实评论误判为虚假评论，所以主要关注classification report里1类的precision和recall：
- 观察可以发现模型的1类里，Precision表现比较好，recall表现相对较差，
提高1类的精确率（Precision for Class 1）: 以确保模型在预测评论为虚假（1类）时更加准确，从而减少将真实评论（0类）错误地分类为虚假评论（1类）的情况。
提高1类的召回率（Recall for Class 1）: 同时，也希望尽可能地识别出所有的虚假评论，这就需要提高1类的召回率，减少虚假评论被遗漏的情况。
但实践中，提高精确率往往会降低召回率，因此要做一个权衡，模型结果1       0.95      0.87      0.90       104， precision比recall高，说明模型优先确保预测虚假评论的准确率，避免误判，符合我们的期望